##### AI TECH - Akademia Innowacyjnych Zastosowań Technologii Cyfrowych. Programu Operacyjnego Polska Cyfrowa na lata 2014-2020
<hr>


In [ ]:
#@title
%%html
<iframe src="https://www.polskacyfrowa.gov.pl/media/48246/FE_POPC_poziom_pl-1_rgb.jpg" width="800"></iframe>


# Uczenie głębokie

Szymon Zaporowski, Politechnika Gdańska, Wydział ETI, Katedra Systemów Multimedialnych

**Wykład 7:** Rekurencyjne Sieci Neuronowe

**Przykład (1):** Wprowadzenie do Rekurencyjnych Sieci Neuronowych


## Tworzenie sieci RNN z wykorzystaniem pakietu PyTorch

W tym notaniku pokazano prosty sposób budowy sieci RNN od zera z wykorzystaniem pakietu PyTorch. W ramach notatnika zostanie stworzona siec składajaca sie z jednej warstwy. Taka prosta konstrukcja pozwoli na prześledzenie jak właściwie zbudowana jest komórka sieci RNN i jak wygląda inicjowanie wag oraz stanów ukrytych.



Wskażmy pakiety, z jakich będziemy korzystać:


In [ ]:
!pip3 install torch torchvision
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import numpy as np

### Sieć RNN z pojedynczym neuronem


Zacznijmy od prostego przykładu, aby zrozumieć mechanizm stojący za działaniem sieci typu RNN.
Na samym początku stwórzmy graf obliczeniowy dla jednowarstwowej sieci RNN.

Poniżej przedstawiono schemat architektury, którą budujemy:
![](https://drive.google.com/uc?export=view&id=127yxax8AuwdBOsWP-Wg-OsZaD4nw2abW)


Poniżej znajduje się kod z implementacją przedstawionej na powyższym rysunku sieci. Wykonajmy komórkę i prześledźmy co własciwie dzieje się w kodzie.

In [ ]:
class  BasicRNN(nn.Module):
    def __init__(self, n_inputs, n_neurons):
        super(BasicRNN, self).__init__()

        n_neurons=n_neurons

        self.Wx = torch.randn(n_inputs, n_neurons) # 4 X n_neurons
        self.Wy = torch.randn(n_neurons, n_neurons) # 1 X n_neurons

        self.b = torch.zeros(1, n_neurons) # 1 X 4

    def forward(self, X0, X1):
        self.Y0 = torch.tanh(torch.mm(X0, self.Wx) + self.b) # 4 X n_neurons

        self.Y1 = torch.tanh(torch.mm(self.Y0, self.Wy) +
                            torch.mm(X1, self.Wx) + self.b) # 4 X n_neurons

        return self.Y0, self.Y1

W powyższek komórce zaimplementowano najprostszą sieć RNN składającą się z jednej warstwy i jednego neurona. Zostały zainicjalizowane dwie macierze wag - `Wx` oraz `Wy` posiadające wartości pochodzące z rozkładu normalnego. `Wx`  zawiera wagi połączeń dla wejścia w obecnym kroku czasowym, podczas gdy `Wy` zawiera wagi połączeń dla wyjść z poprzedniego kroku czasowego. Dodatkowo zastosowano bias ukryty pod zmienną `b` . Funkcja `forward` oblicza dwa wyjścia - po jednym dla każdego kroku czasowego. Zastosowano funkcję tangensa hiperbolicznego `tanh` jako funkcję aktywacji.

Jako wejście wprowadzane są 4 przykłady, gdzie każdy przykład zawiera dwie sekwencje wejściowe.

Poniżej schematycznie przedstawiono w jaki sposób wprowadzane są dane do modelu sieci RNN:
![](https://drive.google.com/uc?export=view&id=18VUYxUyXlo2jbqhHNjfcv8igX1hU38WT)


Wykonajmy poniższą komórkę, aby przetestować model:


In [ ]:
N_INPUT = 4
N_NEURONS = 1

X0_batch = torch.tensor([[0,1,2,0], [3,4,5,0],
                         [6,7,8,0], [9,0,1,0]],
                        dtype = torch.float) #t=0 => 4 X 4

X1_batch = torch.tensor([[9,8,7,0], [0,0,0,0],
                         [6,5,4,0], [3,2,1,0]],
                        dtype = torch.float) #t=1 => 4 X 4
model = BasicRNN(N_INPUT, N_NEURONS)

Y0_val, Y1_val = model(X0_batch, X1_batch)

Po podaniu na wejście grafu obliczeniowego wartości uzyskaliśmy wartości wyjść dla kroków czasowych `Y0` oraz `Y1`. Zobaczmy jak wyglądają wartości tych wyjść.


In [ ]:
print(Y0_val)
print(Y1_val)

tensor([[-0.6834],
        [-0.9635],
        [-0.9963],
        [-0.9984]])
tensor([[-0.9990],
        [-0.9312],
        [-0.9967],
        [-0.9673]])


Wyjścia przyjmują wartość wektora o rozmiarze 4x1, zgodnie ze zdefiniowaną funkcją `forward`

### Zwiększanie liczby neuronów w warstwie sieci RNN
Teraz czas na zwiększenie libczy nueronów w warstwie, tak aby pojedyncza warstwa mogła posiadać liczbę neuronów równą `n` . Należy pamiętać, że w odniesieniu do architektury nie dokonujemy żadnych zmian, gdyż liczba neuronów została już sparametryzowana w grafie obliczeniowym. Jednak zmianabędzie dotyczyła rozmiaru wyjścia, ponieważ został zmieniony rozmiar jednostek (czyli neuronów) w warstwie RNN.

Ponizej ilustracja jak będzie wyglądała taka architektrura:

![](https://drive.google.com/uc?export=view&id=16VQBp3paGSiUgRE6LVUnLoxV8FsTwI4d)


Poniżej przedstawiono kod jak stworzyć architekturę zgodną z powyższą ilustracją, skorzytsamy z wcześniejszej klasy `BasicRNN`, zmienimy liczbę cech wejśćia i neuronów oraz wartość wejść:

In [ ]:
N_INPUT = 3 #  liczba cech w wejściu
N_NEURONS = 4 # liczba jednostek (neuronów) w warstwie
X0_batch = torch.tensor([[0,1,2], [3,4,5],
                         [6,7,8], [9,0,1]],
                        dtype = torch.float) # krok czasowy t=0 => rozmiar 4 X 3

X1_batch = torch.tensor([[9,8,7], [0,0,0],
                         [6,5,4], [3,2,1]],
                        dtype = torch.float) # krok czasowy t=1 => rozmiar 4 X 3

model = BasicRNN(N_INPUT, N_NEURONS)

Y0_val, Y1_val = model(X0_batch, X1_batch)

Teraz, gdy zostaną wyświetlone wartości wyjścia dla każdego kroku czasowego przyjmą one rozmiar `4 X 5` co jest związane odpowiednio z rozmiarem batcha oraz liczbą neuronów



In [ ]:
print(Y0_val)
print(Y1_val)

tensor([[ 0.9700,  0.0453,  0.9774,  0.9987],
        [ 0.9990,  0.5781,  1.0000,  1.0000],
        [ 1.0000,  0.8549,  1.0000,  1.0000],
        [-0.8588, -0.9922,  0.9984, -1.0000]])
tensor([[ 0.9995,  0.6149,  1.0000,  0.9901],
        [ 0.5938, -0.6128,  0.4187, -0.2809],
        [ 0.9504,  0.5702,  1.0000, -0.4382],
        [-0.5133,  0.9162,  0.9921, -0.9553]])


### Bardziej skomplikowana architektura RNN z wykorzystaniem RNNCell z pakiety PyTorch

Zapewne patrząc na wcześniejszy kod można było sobie zadać pewne pytanie. W trakcie wykładu była mowa o tym, że RNN pozwalają na korzystanie z wartości wejść i wyjść, które mają znaczne wymiary. Gdyby budować sieć w pokazany powyżej sposób konieczne byłoby pojedyncze obliczanie wyjścia dla każdego kroku czasowego, tym samym powdoując znaczny przyrost liczby linijek kodu, aby uzyskać zadowalający efekt w grafie obliczeniowym. Poniżej pokazano sposób w jaki można zaimplementować takie podejście z wykorzystaniem modułu RNNCell.

Przeanalizujmy strukturę modułu `RNNCell` :

In [ ]:
rnn = nn.RNNCell(3, 5) # liczba_wejść X liczba_neuronów

X_batch = torch.tensor([[[0,1,2], [3,4,5],
                         [6,7,8], [9,0,1]],
                        [[9,8,7], [0,0,0],
                         [6,5,4], [3,2,1]]
                       ], dtype = torch.float) # X0 and X1

hx = torch.randn(4, 5) # m X liczba_neuronów -> poprzedni krok czasowy
output = []

# dla każdego kroku czasowego
for i in range(2):
    hx = rnn(X_batch[i], hx)
    output.append(hx)

print(output)

[tensor([[ 0.1770,  0.7625, -0.4189,  0.0832, -0.3649],
        [-0.8115,  0.9066, -0.4927,  0.9082, -0.9699],
        [-0.8859,  0.9730, -0.5052,  0.9994, -0.9981],
        [-0.9996, -0.6566, -0.9573,  0.2146, -0.9843]],
       grad_fn=<TanhBackward0>), tensor([[-0.9899,  0.8744, -0.9456,  0.9990, -0.9994],
        [ 0.1901, -0.2070, -0.5764, -0.5149,  0.4377],
        [-0.9605,  0.4545, -0.9505,  0.9586, -0.9760],
        [-0.8177,  0.6691, -0.7424,  0.2129, -0.8660]],
       grad_fn=<TanhBackward0>)]


W powyższej komórce udało się zaimplementować bardzo podobne rozwiązanie jak `BasicRNN` pokazane w 7 komórce. Jednak wykorzystanie `torch.RNNCell(...)` uwalnia nas z konieczności deklarowania macierzy wag i biasów - jest to wykonywane automatycznie za nas. `torch.RNNCell`  akcpetyje tensor jako wejście i wyjscie następnego stanu ukrytego `hx` dla każdego elementu w batchu.

Stwórzmy teraz graf obliczeniowy wykorzstując wiedzę z poprzedniej komórki:


In [ ]:
class CleanBasicRNN(nn.Module):
    def __init__(self, batch_size, n_inputs, n_neurons):
        super(CleanBasicRNN, self).__init__()

        self.rnn = nn.RNNCell(n_inputs, n_neurons)
        self.hx = torch.randn(batch_size, n_neurons) # inicjalizacja ukrytego stanu

    def forward(self, X):
        output = []

        # dla każdego kroku czasowego
        for i in range(2):
            self.hx = self.rnn(X[i], self.hx)
            output.append(self.hx)

        return output, self.hx

In [ ]:
FIXED_BATCH_SIZE = 4 # batch size jest wartością stałą
N_INPUT = 3
N_NEURONS = 5

X_batch = torch.tensor([[[0,1,2], [3,4,5],
                         [6,7,8], [9,0,1]],
                        [[9,8,7], [0,0,0],
                         [6,5,4], [3,2,1]]
                       ], dtype = torch.float) # X0 and X1


model = CleanBasicRNN(FIXED_BATCH_SIZE, N_INPUT, N_NEURONS)
output_val, states_val = model(X_batch)
print('Wartość wyjść:',output_val) # zawiera wszystkie wartości wyjścia dla wszystkich obliczonych kroków czasowych
print("Wartość stanu:",states_val) #zawiera wartość dla finalnego stanu lub kroku czasowego np. t=1

Wartość wyjść: [tensor([[ 0.2094,  0.4826,  0.6720,  0.3811,  0.4934],
        [-0.9826,  0.8380, -0.6931,  0.9694,  0.9147],
        [-0.9998,  0.9849, -0.9526,  0.9991,  0.9972],
        [-0.3999,  0.8226, -0.5932,  0.9996,  0.9992]],
       grad_fn=<TanhBackward0>), tensor([[-1.0000,  0.9915, -0.9990,  1.0000,  0.9998],
        [ 0.0246,  0.1705,  0.2521, -0.5257,  0.5569],
        [-0.9983,  0.9102, -0.9892,  0.9960,  0.9981],
        [-0.8526,  0.4656, -0.7301,  0.8870,  0.9810]],
       grad_fn=<TanhBackward0>)]
Wartość stanu: tensor([[-1.0000,  0.9915, -0.9990,  1.0000,  0.9998],
        [ 0.0246,  0.1705,  0.2521, -0.5257,  0.5569],
        [-0.9983,  0.9102, -0.9892,  0.9960,  0.9981],
        [-0.8526,  0.4656, -0.7301,  0.8870,  0.9810]],
       grad_fn=<TanhBackward0>)


Jak widać kod jest prostszy, nie musimy ręcznie aktualizować wartości macierzy wag - wszystko jest wykonywane automatycznie przez PyTorch.


Zachęcam do eksperymentowania z notatnikiem, prób zwiększania liczby warstw i liczby neuronów, funkcji aktywacji itp.

<center>
Projekt współfinansowany ze środków Unii Europejskiej w ramach Europejskiego Funduszu Rozwoju Regionalnego
Program Operacyjny Polska Cyfrowa na lata 2014-2020,
Oś Priorytetowa nr 3 "Cyfrowe kompetencje społeczeństwa" Działanie  nr 3.2 "Innowacyjne rozwiązania na rzecz aktywizacji cyfrowej"
Tytuł projektu:  „Akademia Innowacyjnych Zastosowań Technologii Cyfrowych (AI Tech)”
    </center>